<a href="https://colab.research.google.com/github/IgorComune/ml_iss/blob/master/iss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#pip install sktime

# Libraries

In [2]:
# Data
import pandas as pd
import numpy as np

# Others
import requests
from time import sleep
import datetime

# data visualizing
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots

# Machine Learning
from sktime.forecasting.naive import NaiveForecaster

# Dataset

In [3]:
iss_main_df = pd.read_csv('iss.csv')
iss_main_df = iss_main_df.drop(columns='Unnamed: 0') #dropping an unwanted column
iss_main_df.head()

,name,id,latitude,longitude,altitude,velocity,visibility,footprint,timestamp,daynum,solar_lat,solar_lon,units
0,iss,25544,-43.998459,163.020987,437.906017,27530.533938,eclipsed,4597.461899,2021-10-13 16:07:45,2.459501e+06,-8.025859,294.602489,kilometers
1,iss,25544,-42.990870,165.160868,437.469412,27531.473009,eclipsed,4595.293095,2021-10-13 16:08:15,2.459501e+06,-8.025988,294.477469,kilometers
2,iss,25544,-41.903966,167.294276,436.995423,27532.500682,daylight,4592.937167,2021-10-13 16:08:46,2.459501e+06,-8.026122,294.348281,kilometers
3,iss,25544,-40.773740,169.351030,436.500023,27533.583581,daylight,4590.473228,2021-10-13 16:09:17,2.459501e+06,-8.026256,294.219093,kilometers
4,iss,25544,-39.603228,171.333950,435.985094,27534.718533,daylight,4587.910440,2021-10-13 16:09:48,2.459501e+06,-8.026389,294.089905,kilometers


# Visualizing the Data

In [4]:
fig = go.Figure(px.line_geo(lat=iss_main_df.latitude, lon=iss_main_df.longitude))
fig.update_traces(marker_size=7, line=dict(color='Red'), opacity=0.9)
fig.update_geos(projection_type="orthographic",lataxis_showgrid=True, lonaxis_showgrid=True, showcountries=True, 
                showland=True, landcolor="Gray", showocean=True, oceancolor="Black")
fig.update_layout(width= 900, height=900, margin={"r":0,"t":0,"l":0,"b":0})
fig.update_geos()
#fig.write_html("3d_plot.html")
fig.show()

In [5]:
fig = go.Figure(px.line_geo(lat=iss_main_df.latitude, lon=iss_main_df.longitude))
fig.update_layout(height=1000, width=1000, margin={"r":0,"t":0,"l":0,"b":0})


fig.update_traces(marker_size=7, line=dict(color='Red'), opacity=0.9)
fig.update_geos(projection_type="natural earth", lataxis_showgrid=True, lonaxis_showgrid=True, showcountries=True, 
                showland=True, landcolor="Gray", showocean=True, oceancolor="Black")

fig.show()

# Predicting ISS Orbit

In [6]:
iss_main_df.head()

,name,id,latitude,longitude,altitude,velocity,visibility,footprint,timestamp,daynum,solar_lat,solar_lon,units
0,iss,25544,-43.998459,163.020987,437.906017,27530.533938,eclipsed,4597.461899,2021-10-13 16:07:45,2.459501e+06,-8.025859,294.602489,kilometers
1,iss,25544,-42.990870,165.160868,437.469412,27531.473009,eclipsed,4595.293095,2021-10-13 16:08:15,2.459501e+06,-8.025988,294.477469,kilometers
2,iss,25544,-41.903966,167.294276,436.995423,27532.500682,daylight,4592.937167,2021-10-13 16:08:46,2.459501e+06,-8.026122,294.348281,kilometers
3,iss,25544,-40.773740,169.351030,436.500023,27533.583581,daylight,4590.473228,2021-10-13 16:09:17,2.459501e+06,-8.026256,294.219093,kilometers
4,iss,25544,-39.603228,171.333950,435.985094,27534.718533,daylight,4587.910440,2021-10-13 16:09:48,2.459501e+06,-8.026389,294.089905,kilometers


In [7]:
iss_main_df.dtypes

name           object
id              int64
latitude      float64
longitude     float64
altitude      float64
velocity      float64
visibility     object
footprint     float64
timestamp      object
daynum        float64
solar_lat     float64
solar_lon     float64
units          object
dtype: object

In [8]:
iss = iss_main_df[['timestamp', 'latitude','longitude']]
iss

,timestamp,latitude,longitude
0,2021-10-13 16:07:45,-43.998459,163.020987
1,2021-10-13 16:08:15,-42.990870,165.160868
2,2021-10-13 16:08:46,-41.903966,167.294276
3,2021-10-13 16:09:17,-40.773740,169.351030
4,2021-10-13 16:09:48,-39.603228,171.333950
...,...,...,...
795,2021-10-13 22:54:21,48.837026,173.342901
796,2021-10-13 22:54:52,49.485493,176.104411
797,2021-10-13 22:55:22,50.042567,178.846232
798,2021-10-13 22:55:53,50.542174,-178.254810


In [9]:
iss['timestamp'] =  pd.to_datetime(iss['timestamp'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
y_lon_train = iss['longitude'][:600]
y_lat_train = iss['latitude'][:600]

y_lon_test = iss['longitude'][600:]
y_lat_test = iss['latitude'][600:]

In [11]:
fh = []

for number in range(200):
  fh.append(number)

In [19]:
forecaster_lat = NaiveForecaster(strategy="drift")
forecaster_lon = NaiveForecaster(strategy="drift")
forecaster_lat.fit(y_lat_train)
forecaster_lon.fit(y_lon_train)

y_pred_lat = forecaster_lat.predict(fh=fh)
y_pred_lon = forecaster_lon.predict(fh=fh)

In [20]:
fig = go.Figure(px.line_geo(lat=y_pred_lat, lon=y_pred_lon))
fig.update_layout(height=1000, width=1000, margin={"r":0,"t":0,"l":0,"b":0})


fig.update_traces(marker_size=7, line=dict(color='Red'), opacity=0.9)
fig.update_geos(projection_type="natural earth", lataxis_showgrid=True, lonaxis_showgrid=True, showcountries=True, 
                showland=True, landcolor="Gray", showocean=True, oceancolor="Black")

fig.show()

In [17]:
y_pred_lat

599   -0.692320
600   -0.634909
601   -0.634909
602   -0.634909
603   -0.634909
         ...   
794   -0.634909
795   -0.634909
796   -0.634909
797   -0.634909
798   -0.634909
Length: 200, dtype: float64

In [18]:
y_lat_test

600    35.081116
601    36.379237
602    37.606216
603    38.841425
604    40.040804
         ...    
795    48.837026
796    49.485493
797    50.042567
798    50.542174
799    50.961520
Name: latitude, Length: 200, dtype: float64